# Background

This notebook is designed to demonstrate how to use Gantry to seamlessly transform 

In [1]:
from config import GantryConfig
import gantry

/Users/oscarbatori/Documents/gantry-intro-tutorial/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
gantry.init(
    api_key=GantryConfig.GANTRY_API_KEY,
    email="test@gantry.dev",
)

assert gantry.ping()

2023-01-25 19:54:24,780 gantry.logger.stores ERROR    Error during api key check: Authentication error. Ensure that you supplied a working API key by calling gantry.ready(). (Status code: 401)
2023-01-25 19:54:24,781 gantry.logger.main WARNING  Gantry services won't receive traffic. Check if API Key is valid
2023-01-25 19:54:25,012 gantry.dataset.client ERROR    Error during api key check: Authentication error. Ensure that you supplied a working API key by calling gantry.ready(). (Status code: 401)
2023-01-25 19:54:25,014 gantry.dataset.main WARNING  Gantry services won't receive traffic. Check if API Key is valid


In [ ]:
from gantry.curators import BalancedStratificationCurator

bsc = BalancedStratificationCurator(
    name = "",
    application_name = "",
    stratify_on = "",
    limit = 10000, 
)

bsc.create()

In [4]:
GantryConfig.GANTRY_API_KEY

'Jji45UcKkXuOOw3JXbRRb2jvF_E'